In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep, strftime
from random import randint
import pandas as pd

In [ ]:
# Use WebDriver to open a Chrome tab and navigate to Instagram login page
chromedriver_path = "/Users/jinwkim/Downloads/chromedriver"
webdriver = webdriver.Chrome(executable_path = chromedriver_path)
webdriver.get("https://www.instagram.com/accounts/login")
sleep(1)

In [ ]:
# Enter username/password to log in
un, pw = "",""
username = webdriver.find_element_by_name("username")
username.send_keys(un)
password = webdriver.find_element_by_name("password")
password.send_keys(pw)
sleep(1)

# Click login button
button_login = webdriver.find_element_by_css_selector("#react-root > section > main > div > article > div > div:nth-child(1) > div > form > div:nth-child(4) > button")
button_login.click()
sleep(3)

In [ ]:
# Click "Not Now" on "Save Your Login Info?" popup
not_now = webdriver.find_element_by_css_selector("#react-root > section > main > div > div > div > div > button")
not_now.click()
sleep(randint(1,2))

# Click "Not Now" on popup "Turn on Notifications"
not_now = webdriver.find_element_by_css_selector("body > div.RnEpo.Yx5HN > div > div > div > div.mt3GC > button.aOOlW.HoLwm")
not_now.click()
sleep(randint(1,2))

In [ ]:
# Initialize variables to keep track of
# already_following = [] # need to track who we're following already 
already_following = pd.read_csv("users_followed_list.csv", delimiter=",").iloc[:,1]
already_following = set(list(already_following)) # use set to get O(1) search if needed

new_followed = []
followed, likes, comments = 0, 0, 0

In [ ]:
# Specify which hashtags you want to explore
hashtag_list = ["Food","FoodPorn","InstaFood"]
# hashtag_list = ["Dinner","Lunch","BestFoodBoston"]
# hashtag_list = ["Foodstagram","FoodPhotography","BuzzFeedFood"]

In [ ]:
# Iterate through the hashtags stored in "hashtag_list"
for hashtag in hashtag_list:
    
    # Navigate to Instagram "explore/tags" page for current hashtag
    webdriver.get("https://www.instagram.com/explore/tags/"+hashtag+"/")
    sleep(randint(1,2))
    
    # Click on the first thumbnail in the current hashtag's explore page
    first_thumbnail = webdriver.find_element_by_css_selector("#react-root > section > main > article > div.EZdmt > div > div > div:nth-child(1) > div:nth-child(1) > a > div > div._9AhH0")
    first_thumbnail.click()
    sleep(randint(1,2))
    
    try:
        # Iterate through first 10 pictures in the current hashtag
        for _ in range(3):
            
            # Capture the username of the user who posted
            username = webdriver.find_element_by_css_selector("body > div._2dDPU.CkGkG > div.zZYga > div > article > header > div.o-MQd.z8cbW > div.PQo_0.RqtMr > div.e1e1d > a").text

            # If we are not currently following the user, click "Follow" => click only if button says "Follow"
            follow = webdriver.find_element_by_css_selector("body > div._2dDPU.CkGkG > div.zZYga > div > article > header > div.o-MQd.z8cbW > div.PQo_0.RqtMr > div.bY2yH > button")
            if follow.text == "Follow" and username.lower() != "eatlikeabos":

                # Click follow
                follow.click()
                sleep(randint(1,3))
                # Increment "followed" counter, add username to new_followed list
                new_followed.append(username)
                followed += 1

                # Click like
                like = webdriver.find_element_by_css_selector("body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > section.ltpMr.Slqrh > span.fr66n > button > svg")
                like.click()
                sleep(randint(1,3))
                # Increment "likes" counter
                likes += 1

                # Post a comment
                # Click into the textarea of comments section
                comment = webdriver.find_element_by_css_selector("body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > section.sH9wk._JgwE > div > form > textarea")
                comment.click()
                sleep(randint(2,4))
                # Use "randint" to post different comments
                rand_comment = randint(1,5)
                comment = webdriver.find_element_by_css_selector("body > div._2dDPU.CkGkG > div.zZYga > div > article > div.eo2As > section.sH9wk._JgwE > div > form > textarea")
                if rand_comment == 1:
                    comment.send_keys("Awesome shot!")
                elif rand_comment == 2:
                    comment.send_keys("Looks delicious! Yum")
                elif rand_comment == 3:
                    comment.send_keys("Wow! This looks amazing!")
                elif rand_comment == 4:
                    comment.send_keys("That looks so tasty!")
                else:
                    comment.send_keys("This looks so delicious! Great shot!")
                sleep(randint(3,5)) # some extra time to edit comment
                # Click enter to post comment
                comment.send_keys(Keys.ENTER)
                sleep(randint(2,4))
                # Increment "comments" counter
                comments += 1
                
            # Click "next" to go to next picture within the same hashtag
            next = webdriver.find_element_by_css_selector("body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow")
            next.click()
            sleep(randint(2,5))
                
    except Exception as ex:
        
        # Write out what type of Exception
        template = "An exception of type {0} occurred. Arguments:\n{1!r}"
        message = template.format(type(ex).__name__, ex.args)
        print(message)
        
        

In [ ]:
# Merge the "new_followed" list to "already_following"
already_following = list(already_following) # cast to list from set
already_following.extend(new_followed)

# Write out the CSV file of new users we are following
updated_user_df = pd.DataFrame(already_following)
updated_user_df.to_csv("users_followed_list.csv")

# Print out stats
print("Liked {} images.".format(likes))
print("Commented on {} images.".format(comments))
print("Followed {} new people.".format(followed))

In [ ]:
"""
creds to Fábio Neves
https://towardsdatascience.com/increase-your-instagram-followers-with-a-simple-python-bot-fde048dce20d?gi=3e7519ab06bf
"""